In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import PIL
import cv2
import torchvision.transforms as transforms
import PIL.Image
import glob


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
import json
import trt_pose.coco
import timm
import time

with open('human_pose_2.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)
topology

tensor([[ 0,  1,  0,  1],
        [ 2,  3,  1,  2],
        [ 4,  5,  2,  3],
        [ 6,  7,  3,  4],
        [ 8,  9,  4,  5],
        [10, 11,  5,  6],
        [12, 13,  2,  4]], dtype=torch.int32)

In [3]:
import trt_pose.models

num_parts = len(human_pose['keypoints'])
num_links = len(human_pose['skeleton'])

model = trt_pose.models.efficientnet_b3_baseline_att(num_parts, 2 * num_links).cuda().eval()

In [4]:
model[0].efficientnet.conv_stem.weight[0, 0]

tensor([[ 0.0642,  0.0589, -0.0545],
        [ 0.1169, -0.2110, -0.4134],
        [ 0.1067,  0.0160, -0.1370]], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [5]:
import torch
MODEL_WEIGHTS = 'epoch_200.pth'
state_dict = torch.load(MODEL_WEIGHTS)

from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
    
model.load_state_dict(new_state_dict)    

<All keys matched successfully>

In [6]:
model[0].efficientnet.conv_stem.weight[0, 0]

tensor([[ 0.4986,  0.1909, -0.2082],
        [ 0.4296, -0.1981, -2.0589],
        [ 0.2448,  0.4187, -0.4783]], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [7]:
SIZE = 320

In [8]:
import torch2trt
# WIDTH = SIZE
# HEIGHT = SIZE
# data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()

# model_trt = torch2trt.torch2trt(model, [data], fp16_mode=False, max_workspace_size=1<<25)
# model_trt = torch2trt.torch2trt(model, [data], fp16_mode=True, max_workspace_size=3000)

OPTIMIZED_MODEL = 'epoch_200_fp16_trt.pth'
# torch.save(model_trt.state_dict(), OPTIMIZED_MODEL)

In [9]:
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

<All keys matched successfully>

In [10]:
# from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects
# from draw import DrawObjects, SaveImage
from utils import GetKeysBoxes, visualize, DrawObjects, get_matching, visualize_goods, personId_matching
import msg_utils

parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)
# save_image = SaveImage(topology)
get_keysboxes = GetKeysBoxes(topology)

[{'shelfNum': 8, 'annotation': [[257, 74, 1, 317, 66, 1, 376, 56, 1, 447, 51, 1, 511, 44, 1, 584, 34, 1], [287, 130, 1, 344, 124, 1, 404, 116, 1, 465, 111, 1, 528, 107, 1, 595, 104, 1], [341, 189, 1, 418, 189, 1, 504, 182, 1, 589, 184, 1], [372, 245, 1, 445, 245, 1, 518, 243, 1, 595, 242, 1], [413, 289, 1, 502, 291, 1, 593, 290, 1]]}, {'shelfNum': 9, 'annotation': [[702, 42, 1, 772, 42, 1, 845, 43, 1, 911, 49, 1, 965, 52, 1, 1030, 64, 1], [695, 110, 1, 757, 112, 1, 815, 116, 1, 878, 121, 1, 938, 128, 1, 996, 133, 1], [703, 181, 1, 789, 187, 1, 877, 192, 1, 944, 192, 1], [699, 242, 1, 772, 240, 1, 844, 243, 1, 915, 242, 1], [703, 289, 1, 790, 289, 1, 878, 290, 1]]}, {'shelfNum': 10, 'annotation': [[1112, 82, 1, 1158, 87, 1, 1202, 93, 1, 1237, 104, 1, 1270, 109, 1, 0, 0, 0], [1070, 143, 1, 1111, 144, 1, 1150, 150, 1, 1196, 158, 1, 1232, 165, 1, 1264, 173, 1], [1031, 195, 1, 1097, 205, 1, 1152, 209, 1, 1203, 218, 1], [1016, 248, 1, 1055, 251, 1, 1116, 252, 1, 1167, 260, 1], [984, 292, 1, 

In [11]:
from sort import *

In [12]:
np.random.randint(0, 200, 1)[0]

172

In [13]:
# mot_tracker = Sort(max_age=5, iou_threshold=0.15) #create instance of the SORT tracker
mot_tracker = Sort() #create instance of the SORT tracker

In [14]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
device = torch.device('cuda')


# cap = cv2.VideoCapture('./video/video_sample_34_1.mp4')
# cap = cv2.VideoCapture('./video/2person_35.h264')
cap = cv2.VideoCapture('./video/out_3.mp4')

pTime = 0
img_array = []
count = 0

persons = {}

# need to keep person's goods
person_goods = {}

while True :
    success, img = cap.read()

    if success == False :
        break

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resized_image = cv2.resize(img, (SIZE,SIZE), interpolation=cv2.INTER_NEAREST)
    
    image = resized_image / 255.
    image = (image - mean[None, None, :])/(std[None, None, :])
    image = torch.tensor(image, dtype=torch.float32).permute(2,0,1)
    image = image.to(device)    
    
    cmap, paf = model_trt(image[None, ...])
    # cmap, paf = model(image[None, ...])
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf) 
    boxes, keys = get_keysboxes(resized_image, counts, objects, peaks)
    
    # goods_point = get_goods_point()
    # goods = barcode_to_goods(barcode)
    # msg = consumer.poll(0.001)
    kmsg = msg_utils.kafka_pog_msg()
    
    goods_point = None
    goods = None
    
    if kmsg != None :
        shelf_num, shelf_row, col, barcode, qty = kmsg
        if shelf_num == 5:
            shelf_num = 10
        if shelf_num == 6:
            shelf_num = 9
        if shelf_num == 7:
            shelf_num = 8    
            
        if shelf_num < 5 :
            cam_num = 0
        else :
            cam_num = 1
        print(msg_utils.pd_coord(0, 2, 2, 3))
        goods_point = msg_utils.pd_coord(cam_num, shelf_num, shelf_row, col)
        print(goods_point)
        # goods = barcode_to_goods[barcode]

    if len(boxes) > 0  :
        # print('*****************************************************************************')
        boxes = np.array(boxes)
        # print(boxes)
        prob = np.ones((boxes.shape[0], 1)).astype(np.float32)
        boxes1 = np.c_[boxes, prob]
        trackers = mot_tracker.update(boxes1).astype(np.uint)
        # print(trackers)
        
        m = iou_batch(boxes, trackers[:,:4])
        # print(m)
        index = np.argmax(m, axis=0)
        m_keys = [keys[i] for i in index]
        
        persons = dict([(key, m_keys[i]) for i, key in enumerate(trackers[:,4])])
        # print(person)
        # goods_point = get_goods_point()
        # goods = barcode_to_goods(barcode)
        
        if goods_point != None and goods != None and len(trackers) > 0 :
            # print(persons)
            person_id, nearest_body = get_matching(persons, goods_point)
            # print(person_id)
            # print(trackers[:,4])
            
            # print(person_id)
            new, remove, add = personId_matching(trackers[:,4], list(person_goods.keys()))
            
            if len(new) > 0 :
                for n in new :
                    if person_id == n :
                        person_goods[person_id] = {'point': nearest_body, "goods": [goods] }
                    else :
                        person_goods[person_id] = {}
                    
            if len(add) > 0 :
                for a in add :
                    if person_id == a :
                        if 'goods' in person_goods[a].keys() :
                            person_goods[person_id]['goods'].append(goods)
                            person_goods[person_id]['point'] = nearest_body
                        else :
                            person_goods[person_id] = {'point': nearest_body, "goods": [goods] }
                    # else :
                    #     pass
                        
            if len(remove) > 0:
                for rm in remove :
                    del person_goods[rm]

            # if person_id in person_goods.keys() :
            #     person_goods[person_id]['point'] = nearest_body
            #     person_goods[person_id]['goods'].append(goods)
            # else :
            #     person_goods[person_id] = {'point': nearest_body, "goods": [goods] }
            
            # print(trackers[:,4])
            # print(person_goods)
             
        # resized_image = visualize(resized_image, boxes1[:,:4], boxes1[:,4])
        # resized_image = visualize(resized_image, trackers[:,:4], trackers[:,4])
    # resized_image = draw_objects(resized_image, counts, objects, peaks, 'test')
    resized_image = visualize_goods(resized_image, person_goods)
        
    img_array.append(resized_image)

out = cv2.VideoWriter('./video/output.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 4, (SIZE, SIZE))
for i in range(len(img_array)) :
    out.write(img_array[i])
    
out.release()    

print('done')

[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None
[]
None


OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


done


In [15]:
# list(person_goods.keys())
a = [1, 2, 3]
a.remove(1)
a

[2, 3]

In [16]:
persons = {24.0: [[119, 147], [106, 165], [119, 194]], 21.0: [[235, 101], [249, 99], [213, 105]], 17.0: [[261, 251], [285, 231], [270, 226]]}
goods_point = (125, 210)

In [17]:
def get_matching(persons, goods_point) :  
    ids = list(persons.keys())
    diff = [ [ np.sqrt((goods_point[0] - p[0])**2 + (goods_point[1] - p[1])**2)  for p in points ]for points in persons.values()]
    
    index = [ min(d) for d in diff]
    index_i = index.index(min(index))
    index_y = diff[index_i].index(min(diff[index_i]))    
    nearest_p = persons[ids[index_i]][index_y]

    return ids[index_i], nearest_p
    
get_matching(persons, point)    

NameError: name 'point' is not defined

In [ ]:
bb_test = [[201.,  98., 249., 129.],
 [107., 133., 134., 194.],
 [  2., 190.,  62., 226.],
 [247., 199., 287., 250.]]
# [21. 17.]
bb_gt = [[201,  96, 248, 130],
 [247, 200, 285, 252],
        [  2., 190.,  62., 226.]]
bb_test = np.array(bb_test)
bb_gt = np.array(bb_gt)

In [ ]:
m = iou_batch(bb_test, bb_gt)
m

In [ ]:
np.argmax(m, axis=0)

In [ ]:
topology

In [ ]:
objects.shape

In [ ]:
objects[0,:5,:]

In [ ]:
peaks.shape

In [ ]:
print('done')

In [ ]:
# ls '../../../images'

In [ ]:
img = cv2.imread('../../../images/' + '694.jpg')
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
# plt.show()

In [ ]:
plt.imshow(cmap[0,0,:,:,])

In [ ]:
img = cv2.imread('../../../images/' + '694.jpg')
SIZE = 320
resized_image = cv2.resize(img, (SIZE,SIZE), interpolation=cv2.INTER_NEAREST)
image = resized_image / 255.
image = (image - mean[None, None, :])/(std[None, None, :])

image = torch.tensor(image, dtype=torch.float32).permute(2,0,1)
image = image.to(device)   

cmap, paf = model(image[None, ...])
cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
counts, objects, peaks = parse_objects(cmap, paf)  
image = draw_objects(resized_image, counts, objects, peaks)
plt.imshow(image)

In [ ]:
plt.imshow(paf[0,0,:,:,])

In [ ]:
plt.imshow(cmap[0,0,:,:,])

In [ ]:
plt.imshow(cmap[0,6,:,:,])

In [ ]:
plt.imshow(paf[0,0,:,:,])

In [ ]:
plt.imshow(paf[0,1,:,:,])

In [ ]:
plt.imshow(paf[0,10,:,:,])

In [ ]:
plt.imshow(paf[0,11,:,:,])